![](https://fenc.wang/imgs/2020/08/3565654bfbe9ffa0.png)

In [1]:
import tushare as ts
import pandas as pd

#设置完整显示数据
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

pro = ts.pro_api('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
cn_en={
    'end_date':'报告期',
    'total_mv':'总市值 （万元）',
    'ebitda':'息税折旧摊销前利润',
    'lt_borr':'长期借款',
    'st_borr':'短期借款',
    'money_cap':'货币资金',
    'trad_asset':'交易性金融资产'
}

#股票池
stock_list = ['爱旭股份', '晶澳科技', '协鑫集成', '中环股份', '东方日升', '中来股份', '亿晶光电', '天合光能', '向日葵', '隆基股份', '通威股份', '正泰电器', '航天机电', '拓日新能']

## 主要用于获取代码对应的中文名称

In [2]:
stock_basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
stock_selections = stock_basic[stock_basic['name'].isin(stock_list)]
stock_selections

,ts_code,name
587,002129.SZ,中环股份
675,002218.SZ,拓日新能
911,002459.SZ,晶澳科技
957,002506.SZ,协鑫集成
1570,300111.SZ,向日葵
1577,300118.SZ,东方日升
1846,300393.SZ,中来股份
2571,600151.SH,航天机电
2804,600438.SH,通威股份
2878,600537.SH,亿晶光电


# 公式：EV=MC+Total Debt−C
### 下面要注意的是：财务数据为2020年的，有滞后性； 市值`total_mv`截至到2021年07月15日；

In [3]:
df = pd.DataFrame()
gf = pd.DataFrame()
for code in stock_selections['ts_code']:
    ebitda = pro.fina_indicator(ts_code=code,period='20201231',fields='ts_code,end_date,ebitda')
    debt_c = pro.balancesheet(ts_code=code,period='20201231',fields='ts_code,end_date,lt_borr,st_borr,money_cap,trad_asset')
    gf=pd.merge(ebitda,debt_c,on=['ts_code','end_date'])
    mv = pro.daily_basic(trade_date='20210715', fields='ts_code,total_mv')
    mv['total_mv'] = mv['total_mv']*10000
    gf=gf.merge(mv)
    df= df.append(gf)
df.drop_duplicates(subset=['ts_code','end_date'],keep='last',inplace=True)
df.set_index('end_date', inplace=True)


In [5]:
#获取股票代码对应的中文名称
df=df.merge(stock_selections, on=['ts_code'])
df.head()

,ts_code,ebitda,money_cap,trad_asset,lt_borr,st_borr,total_mv,name
0,002129.SZ,"4,830,394,685.03","7,515,854,433.95","374,535,025.80","9,225,755,188.49","1,692,000,000.00","136,360,377,328.00",中环股份
1,002218.SZ,"614,376,132.35","510,638,337.64",None,"208,969,421.34","1,123,000,000.00","7,010,059,730.00",拓日新能
2,002459.SZ,"4,102,899,731.25","9,492,867,871.01",None,"777,574,540.32","4,631,170,200.95","84,658,226,679.00",晶澳科技
3,002506.SZ,"-1,813,653,560.32","3,447,321,015.12",0.0,None,"2,171,868,774.35","22,309,465,161.00",协鑫集成
4,300111.SZ,"90,263,312.66","225,642,535.14",None,None,None,"3,471,380,000.00",向日葵


## 这里是计算EV和EV/EBITDA

In [6]:
df = df.fillna(0)
df['ev'] = df['total_mv']+df['lt_borr']+df['st_borr']-df['money_cap']-df['trad_asset']
df['ev/ebitda'] = df['ev'] / df['ebitda']

## 筛选EV和EV/EBITDA 大于0的股票（可选项）

In [8]:
#df_filter = df[(df[['ev', 'ev/ebitda']] > 0).all(1)]
df.sort_values(by=['ev/ebitda'],inplace=True)
df = df.rename(columns=cn_en)
df

,ts_code,息税折旧摊销前利润,货币资金,交易性金融资产,长期借款,短期借款,总市值 （万元）,name,ev,ev/ebitda
3,002506.SZ,"-1,813,653,560.32","3,447,321,015.12",0.00,0.00,"2,171,868,774.35","22,309,465,161.00",协鑫集成,"21,034,012,920.23",-11.60
9,600537.SH,"-322,958,950.10","1,993,500,182.34",0.00,"24,000,000.00","60,000,000.00","5,328,907,484.00",亿晶光电,"3,419,407,301.66",-10.59
12,601877.SH,"10,334,268,531.29","5,976,128,282.81","15,811,063.20","12,327,584,249.86","837,723,601.34","69,293,647,549.00",正泰电器,"76,467,016,054.19",7.40
5,300118.SZ,"1,491,318,404.43","5,167,824,832.94","638,412,772.27","1,715,991,806.77","3,547,361,295.59","17,486,382,855.00",东方日升,"16,943,498,352.15",11.36
1,002218.SZ,"614,376,132.35","510,638,337.64",0.00,"208,969,421.34","1,123,000,000.00","7,010,059,730.00",拓日新能,"7,831,390,813.70",12.75
2,002459.SZ,"4,102,899,731.25","9,492,867,871.01",0.00,"777,574,540.32","4,631,170,200.95","84,658,226,679.00",晶澳科技,"80,574,103,549.26",19.64
10,600732.SH,"1,580,387,795.81","1,254,711,684.25","5,184,124.05","978,811,167.04","1,152,595,500.00","30,198,761,843.00",爱旭股份,"31,070,272,701.74",19.66
7,600151.SH,"616,020,105.08","999,519,108.76","4,198,324.20","617,365,296.88","1,178,053,558.47","13,051,695,812.00",航天机电,"13,843,397,234.39",22.47
8,600438.SH,"7,377,596,387.66","6,264,168,242.03","1,531,863,068.12","6,296,585,539.34","2,349,154,525.77","181,997,593,079.00",通威股份,"182,847,301,833.96",24.78
13,688599.SH,"2,907,855,950.42","9,429,357,046.49","914,709,238.85","4,108,253,652.66","5,067,682,762.45","73,414,936,313.00",天合光能,"72,246,806,442.77",24.85


作者：柯西君_BingWong

网站：cnVar.cn